In [197]:
import pandas as pd
import chardet
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

# ディレクトリ抽出
f = os.path.abspath("./attribute")
files = os.listdir(f)

#ファイル名
list = []
for i in files:
    if "csv" in i:
        list.append(i)        

concat_list = []

# 下処理
for j in list:
    valiables = ['参照元/メディア パス', 'コンバージョン数']
    file = 'attribute/' + j
    date = j.split('.')
    
#     csv読込と年月追加
    df = pd.read_csv(file, header=5)
    df_attribute = df[valiables]
    df_date = df_attribute.assign(date = date[0])
    concat_list.append(df_date)

#concat
df = pd.concat(concat_list,  ignore_index=True)

In [199]:
# アトリビュート経路リスト格納
evaluation_list = df['参照元/メディア パス'].str.split('>')
count_list = [[],[],[]]

#ファーストタッチとラストタッチの評価
#いつか中間貢献度も出す
for i in range(len(evaluation_list)):
    count_list[2].append(len(evaluation_list[i]))    #CV者特定期間の回数
    
    if("cpc" in evaluation_list[i][0] or "display" in evaluation_list[i][0]) :    #  ファーストタッチ
        count_list[0].append(1)
    else:
        count_list[0].append(0)
    if ("cpc" in evaluation_list[i][len(evaluation_list[i])-1] or "cpc" in evaluation_list[i][len(evaluation_list[i])-1]): #  ラストタッチ 
        count_list[1].append(1)
    else:
        count_list[1].append(0)

#ファーストタッチとラストタッチ/データ追加
df['first_touch'] = count_list[0]
df['last_touch'] = count_list[1]
df['count_touch'] = count_list[2]

# 抽出条件
df_first = df[df['first_touch'] >=1]
df_first = df_first[df_first['last_touch'] ==0]

# ピポットデータの取得
value = ["コンバージョン数", "count_touch",'first_touch','last_touch']
index = "参照元/メディア パス"
column = "date"

df_setpivot = df_first.pivot_table(values = value, index = index, columns = column, aggfunc = 'sum')

In [ ]:
# csv書き出し
# df_setpivot.to_csv(f + "output_data2.csv")